In [2]:
%pip install gdown
import gdown

url = 'https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM'
output = 'GoogleNews-vectors-negative300.bin'

gdown.download(url, output, quiet=False)

Note: you may need to restart the kernel to use updated packages.
Access denied with the following error:



 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM 



In [4]:
import gensim
from gensim.models import KeyedVectors

# Define the location of the downloaded Word2Vec binary file and the limit
location = 'C:/Users/Ayush Soni/Downloads/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'
limit = 1000000

# Load the Word2Vec model from the binary file with the specified limit
wv = KeyedVectors.load_word2vec_format(location, binary=True, limit=limit)

# Save the Word2Vec model as a flat file
wv.save_word2vec_format('vectors(1).csv')

In [5]:
def get_phrase_embeddings(phrase, wv):
    words = phrase.split()
    embeddings = [wv[word] for word in words if word in wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        return None  # Handle cases where none of the words have embeddings

# Example usage:
phrase = "example phrase"
phrase_embedding = get_phrase_embeddings(phrase, wv)


In [6]:
import chardet

# Detect the encoding of your file
with open('phrases.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Print the detected encoding
print(result['encoding'])

import pandas as pd

# Read the CSV file with error handling
data = pd.read_csv('phrases.csv', encoding='Windows-1252')

Windows-1252


In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

# Load phrases from 'phrases.csv'
phrases_df = data
phrases = phrases_df['Phrases'].tolist()

display(phrases)

['how company compares to its peers?',
 'what is the detailed income statement breakdown of Axa?',
 'world premium penetration in 2020',
 'How does the forecasted insurance premium penetration in country trend compare to its peers?',
 'what are the total losses for companies in country  non life market?',
 'Insurance premiums market in Country',
 'How have the profit margins been for the airlines industry through the years, by regions?',
 'What is the complete profile of the top oil rigs?',
 'what is company general information?',
 'How is the airline industry performing globally?',
 'how does economic profit for in country compare to others?',
 'most profitable insurance company India',
 'Give me a detailed breakup of the income and expenses in South-East Asia',
 'Show me all the oil rigs in the world',
 'What were the premiums earned by the Indian insurance industry in 2020?',
 'How much have premiums grown as compared to claims for India?',
 'Which are the best performing insurance 

In [9]:
# Calculate L2 (Euclidean) distances
phrase_embeddings = [get_phrase_embeddings(phrase, wv) for phrase in phrases]
distances_l2 = euclidean_distances(phrase_embeddings, phrase_embeddings)

# Calculate Cosine distances
distances_cosine = cosine_distances(phrase_embeddings, phrase_embeddings)
display(distances_l2)
display(distances_cosine)

array([[0.0000000e+00, 1.3331060e+00, 1.6539644e+00, ..., 1.4554999e+00,
        1.2601024e+00, 1.5344033e+00],
       [1.3331060e+00, 4.2146848e-08, 1.5864531e+00, ..., 1.2457829e+00,
        1.2353045e+00, 1.2209918e+00],
       [1.6539644e+00, 1.5864531e+00, 0.0000000e+00, ..., 1.4626017e+00,
        1.4938005e+00, 1.5217372e+00],
       ...,
       [1.4554999e+00, 1.2457829e+00, 1.4626017e+00, ..., 0.0000000e+00,
        1.1309116e+00, 8.8096464e-01],
       [1.2601024e+00, 1.2353045e+00, 1.4938005e+00, ..., 1.1309116e+00,
        0.0000000e+00, 1.0785913e+00],
       [1.5344033e+00, 1.2209918e+00, 1.5217372e+00, ..., 8.8096464e-01,
        1.0785913e+00, 0.0000000e+00]], dtype=float32)

array([[0.        , 0.48511565, 0.6306617 , ..., 0.5808232 , 0.43262547,
        0.65011823],
       [0.48511565, 0.        , 0.6882394 , ..., 0.53877676, 0.53459084,
        0.52022624],
       [0.6306617 , 0.6882394 , 0.        , ..., 0.58025837, 0.6100078 ,
        0.632179  ],
       ...,
       [0.5808232 , 0.53877676, 0.58025837, ..., 0.        , 0.44645047,
        0.2698568 ],
       [0.43262547, 0.53459084, 0.6100078 , ..., 0.44645047, 0.        ,
        0.4082259 ],
       [0.65011823, 0.52022624, 0.632179  , ..., 0.2698568 , 0.4082259 ,
        0.        ]], dtype=float32)

In [16]:
def find_closest_match(user_input, phrases, wv, metric='cosine'):
    user_embedding = get_phrase_embeddings(user_input, wv)
    if user_embedding is None:
        return None, None  # No valid embeddings for user input
    distances = []
    for phrase in phrases:
        phrase_embedding = get_phrase_embeddings(phrase, wv)
        if phrase_embedding is not None:
            if metric == 'l2':
                distance = np.linalg.norm(user_embedding - phrase_embedding)
            elif metric == 'cosine':
                distance = np.dot(user_embedding, phrase_embedding)
            distances.append((phrase, distance))
    distances.sort(key=lambda x: x[1])  # Sort by distance
    closest_match, closest_distance = distances[0]
    return closest_match, closest_distance

# Example usage:
user_input = "what"
closest_match, distance = find_closest_match(user_input, phrases, wv, metric='cosine')
display(closest_match)

'Mergers among airline companies?'

In [18]:
def find_closest_match(user_input, phrases, wv, metric='cosine'):
    user_embedding = get_phrase_embeddings(user_input, wv)
    if user_embedding is None:
        return None, None  # No valid embeddings for user input
    distances = []
    for phrase in phrases:
        phrase_embedding = get_phrase_embeddings(phrase, wv)
        if phrase_embedding is not None:
            if metric == 'l2':
                distance = np.linalg.norm(user_embedding - phrase_embedding)
            elif metric == 'cosine':
                distance = np.dot(user_embedding, phrase_embedding)
            distances.append((phrase, distance))
    distances.sort(key=lambda x: x[1])  # Sort by distance
    closest_match, closest_distance = distances[0]
    return closest_match, closest_distance

# Example usage:
user_input = "how"
closest_match, distance = find_closest_match(user_input, phrases, wv, metric='cosine')
display(closest_match)

'world premium penetration in 2020'